In [2]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import json
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [14]:
url = 'https://pro.rbc.ru/interest/finance'
links = []
driver = webdriver.Chrome(executable_path="C:\\Users\\дом.DESKTOP-A3U9TO1\\DataspellProjects\\moretech_prokhodilimimo\\chromedriver\\chromedriver.exe")

try:
    driver.get(url=url)
    time.sleep(1)
    print(1)
    elem = driver.find_element(By.TAG_NAME, "body")
    print(elem)
    no_of_pagedowns = 200
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
        no_of_pagedowns-=1
    html = driver.page_source

    soup = BeautifulSoup(html, 'lxml')

except Exception as e:
    print(e)
finally:
    driver.close()
    driver.quit()


C:\Users\дом.DESKTOP-A3U9TO1\AppData\Local\Temp\ipykernel_13876\2946590815.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\\Users\\дом.DESKTOP-A3U9TO1\\DataspellProjects\\moretech_prokhodilimimo\\chromedriver\\chromedriver.exe")


1
<selenium.webdriver.remote.webelement.WebElement (session="ea5337a860bb361bc27da374fe8906d3", element="40344daf-919b-419b-b291-2833b652b286")>


In [19]:
news_links = soup.find_all('a', {'class' : ['item__link', ]})

In [21]:
#print(soup)
def get_soup(url):
    r = requests.get(url).text
    return BeautifulSoup(r, 'lxml')


def save_json(data):
    with open('rbk_data_fin.json', "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def main():
    news_data = {}

    for i in range(2):

        link = news_links[i].get('href').split('?')[0]
        #print(link)
        name = link
        news_data[name] = {}
        soup = get_soup(link)
        #print(soup.text)

        # Переходим на страницу для дальнейшенго парсинга
        article = soup.find('div', class_='article')
        #print(article)

        #print(article.find('a'))
        #category = article.find('a', class_='article__header__category')
        #date = article.find('span', class_='atricle__date-update').get('content').replace('T', ' ').split('+')[0]
        try:
            date = soup.find('div', class_='atricle__date-update').get_text().strip().split()[1]
        except AttributeError:
            continue
        #print(date)
        title = soup.title
        #image = article.find('div', class_='article__main-image')
        try:
            article_paragraphs = article.find_all('p')
        except AttributeError:
            continue
        article_text = ''
        for paragraph in article_paragraphs:
            article_text += paragraph.text
        #print(article_text)

        # Заполняем полученными данными news_data
        news_data[name]['link'] = link
        news_data[name]['date'] = date
        news_data[name]['text'] = article_text.replace('\xa0', '').replace('\n', ' ').replace('\r', ' ')

        try:
            news_data[name]['title'] = title.text
        except AttributeError:
            news_data[name]['title'] = 'Без заголовка'


    save_json(news_data)
    print("finished")


if __name__ == "__main__":
    main()

<div class="article g-relative js-rbcslider-article">
<div class="l-table">
<div class="l-col-main" data-io-article-url="https://pro.rbc.ru/news/633c17b59a7947a5f193efd6">
<div class="l-col-center-590 article__content">
<div class="plashka-trial js-plashka-trial" style="display: none;">
<a class="plashka-trial__button js-plashka-trial-button js-yandex-counter js-ga-counter" data-ga-event="Button-Click on plate (mobile)" data-mixpanel-link="Click on plate (mobile)" data-yandex-name="Click on plate (mobile)" href="/offers">
        Подписаться на Pro за 1 ₽
    </a>
</div> <div class="article__access">
<div class="article__access__left">
<span class="g-icon-section g-icon-section_rbcpro medium light"></span>
<div class="article__access__text">Материал раздела <b>Основной</b></div>
</div>
<div class="article__access__right">
<div class="article__header__share-block js-social-toggle">
<div class="article__header__share-block__pseudo js-social-toggle-close"></div>
<div class="article__heade